In [1]:
import torchtext
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torchtext.data import Field,LabelField
from torchtext.data import Example
from sklearn.model_selection import train_test_split
from torchtext import data

Inspired by :
    https://www.kaggle.com/swarnabha/pytorch-text-classification-torchtext-lstm

# Tokenizer

In [2]:
import spacy
spacy = spacy.load("en_core_web_sm")
# nltk.download('punkt')

# can't get spacy to work. 
# How do we get it so we can get rid of puntuation and stuff
# Make sure you get rid of stop words somewhere. 

from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")

#tokens = tokenizer(df['text'][1])
#tokens

In [3]:
df= pd.read_csv("torchtext_data/train.csv")
df = df.rename(columns = {'Script':'text','Labels':'target'})
df = df[0:10]
df

,target,text
0,8,"\t\t\tCrouching Tiger, Hidden Dragon\n\n\t\t\t..."
1,4,"""MUMFO..."
2,6,MAX PAYNE\n\n ...
3,6,SLUMDOG MILLIONAIRE\n\n ...
4,16,<b><!--\n\n</b>if (window!= top)\n\ntop.locati...
5,15,THE OTHER BOLEYN GIR...
6,19,GET CARTER...
7,15,"CRAZY, STUPID, LOV..."
8,1,TRISTAN + ...
9,6,BADLANDS\n\nby Terence Malick\n\nFinal Version...


In [4]:
# Complete some basic text cleaning.
# to clean data
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [5]:
df['text'] = normalise_text(df['text'])

In [6]:
train_df, valid_df = train_test_split(df)

In [7]:
valid_df.shape

(3, 2)

In [8]:
# tokenizer = "spacy" uses spacy's tokenizer
# Option to add in sequential = True here. Look into this. 
TEXT = Field(tokenize=tokenizer, include_lengths = True)
LABEL = LabelField(dtype=torch.long, sequential=False)

In [88]:
def create_dataset(df):
    # Do we need to add one example at a time.
    fields = [('text',TEXT),('label', LABEL)]
    examples_prep = []
    for i, row in df.iterrows():
        label = row.target
        text = row.text
        examples_prep.append(Example.fromlist([text,label], fields))

    dataset_output = torchtext.data.Dataset(examples = examples_prep, fields=fields)
    return dataset_output

In [89]:
train_ds = create_dataset(train_df)
valid_ds = create_dataset(valid_df)

In [90]:
#Check an example: 
#print(vars(pytorch_text_dataset[2]))
#print(type(pytorch_text_dataset[2]))

In [91]:
# build the text vocab. 
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_ds, max_size= MAX_VOCAB_SIZE, vectors = 'glove.6B.200d',unk_init = torch.Tensor.zero_)

In [92]:
# Why do we build vocab for the label
LABEL.build_vocab(train_ds)

In [93]:
# Build an iterator
BATCH_SIZE = 1

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_ds, valid_ds), 
    batch_size = BATCH_SIZE,)

In [96]:
for batch in train_iterator:
    print((batch.text[0]))
    break

tensor([[1056],
        [1468],
        [ 194],
        ...,
        [  26],
        [   2],
        [ 248]])


In [35]:
valid_iterator

<bound method Iterator.data of <torchtext.data.iterator.BucketIterator object at 0x000001F6B35A3080>>

In [108]:
vocab_size = len(TEXT.vocab)
vocab_size
embedding_dim = 200

In [110]:
# Need to build my model

class classification_net(nn.Module):
    
    def __init__(self,vocab_size,embedding_dim):
        super(classification_net, self).__init__()
        
        self.main = nn.Sequential(
        nn.Embedding(vocab_size, embedding_dim),
        nn.Linear(embedding_dim, 22),
        nn.Softmax()
        )
    def forward(self, input):
        output = self.main(input)
        return output

In [111]:
network = classification_net(vocab_size, embedding_dim)

In [10]:
network

NameError: name 'network' is not defined

In [9]:
network.embedding

NameError: name 'network' is not defined

In [ ]:
# Load the pretrained vectors into the embedding matrix.
pretrained_embeddings = TEXT.vocab.vectors

network.embedding.weight